In [3]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from category_encoders import TargetEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV


In [18]:
df = pd.read_csv('/Users/macbookpro/Documents/Documents/open_project/compe/Intelecta/train_cleaned.csv')
df.head()

,ID,Tahun,Nama_Negara,Wilayah,Jenis_Tanaman,Total_Curah_Hujan_mm,Emisi_CO2_JT_Ton,Hasil_Panen_Ton_per_HA,Kejadian_Cuaca_Ekstrim,Akses_Irigasi,Penggunaan_Pestisida_KG_per_HA,Penggunaan_Pupuk_KG_per_HA,Indeks_Kesehatan_Tanah,Strategi_Adaptasi,Suhu_Rata_Rata_C
0,0,2015,USA,South,Soybeans,1658.71,13.36,2.620,10,74.41,38.97,2.64,46.07,Manajemen Air,20.43
1,1,2022,China,East,Wheat,1478.74,9.55,0.570,2,36.90,49.99,77.22,88.87,Rotasi Tanaman,-0.33
2,2,2000,India,West Bengal,Fruits,1252.34,27.37,2.115,3,34.21,2.75,83.94,77.15,Pertanian Organik,12.97
3,3,2008,Nigeria,North West,Sugarcane,209.89,16.16,4.158,5,91.74,36.80,37.50,73.59,Pertanian Organik,12.81
4,4,1991,Canada,Ontario,Vegetables,1086.67,3.71,2.430,0,14.72,7.22,28.72,41.90,Tanpa Adaptasi,4.22


In [19]:
df_test = pd.read_csv('/Users/macbookpro/Documents/Documents/open_project/compe/Intelecta/test_cleaned.csv')
df_test.head()

,ID,Tahun,Nama_Negara,Wilayah,Jenis_Tanaman,Total_Curah_Hujan_mm,Emisi_CO2_JT_Ton,Hasil_Panen_Ton_per_HA,Kejadian_Cuaca_Ekstrim,Akses_Irigasi,Penggunaan_Pestisida_KG_per_HA,Penggunaan_Pupuk_KG_per_HA,Indeks_Kesehatan_Tanah,Strategi_Adaptasi
0,8000,2022,Russia,Central,Fruits,1173.04,10.29,2.440,3,59.17,29.17,80.06,74.93,Tanpa Adaptasi
1,8001,1993,Brazil,South,Rice,310.08,13.90,3.276,2,60.59,4.76,70.63,97.59,Tanpa Adaptasi
2,8002,1998,India,Maharashtra,Soybeans,1348.17,12.90,2.620,3,90.93,5.95,63.87,74.81,Tanaman Tahan Kekeringan
3,8003,2010,Russia,Volga,Soybeans,1732.43,23.82,3.717,4,35.11,7.49,81.81,87.01,Rotasi Tanaman
4,8004,1998,Canada,British Columbia,Sugarcane,2242.77,28.92,2.034,6,68.75,43.35,48.14,64.22,Manajemen Air


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 15 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ID                              8000 non-null   int64  
 1   Tahun                           8000 non-null   int64  
 2   Nama_Negara                     8000 non-null   object 
 3   Wilayah                         8000 non-null   object 
 4   Jenis_Tanaman                   8000 non-null   object 
 5   Total_Curah_Hujan_mm            8000 non-null   float64
 6   Emisi_CO2_JT_Ton                8000 non-null   float64
 7   Hasil_Panen_Ton_per_HA          8000 non-null   float64
 8   Kejadian_Cuaca_Ekstrim          8000 non-null   int64  
 9   Akses_Irigasi                   8000 non-null   float64
 10  Penggunaan_Pestisida_KG_per_HA  8000 non-null   float64
 11  Penggunaan_Pupuk_KG_per_HA      8000 non-null   float64
 12  Indeks_Kesehatan_Tanah          80

In [20]:
df = df.drop(columns=['ID'])

In [27]:
# ==== Feature Engineering untuk Training ====
from sklearn.base import TransformerMixin, BaseEstimator


class FeatureEngineeringTrain(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        df = X.copy()
        
        # Kolom gabungan wilayah
        df['Negara_Wilayah'] = df['Nama_Negara'] + '_' + df['Wilayah']

        # Representasi waktu linear
        df['Laju_Pertumbuhan_Tahun'] = df['Tahun'] - df['Tahun'].min()
        
        # Dekade
        df['Dekade'] = (df['Tahun'] // 10) * 10

        # Perubahan hasil panen dan suhu per tahun (lag 1)
        df['Delta_Hasil_Panen'] = df.groupby(['Nama_Negara', 'Jenis_Tanaman'])['Hasil_Panen_Ton_per_HA'].diff()
        df['Perubahan_Suhu_Tahunan'] = df.groupby(['Nama_Negara'])['Suhu_Rata_Rata_C'].diff()

        # Moving average hasil panen 3 tahun terakhir
        df['Moving_Avg_Hasil_Panen_3Tahun'] = (
            df.groupby(['Nama_Negara', 'Jenis_Tanaman'])['Hasil_Panen_Ton_per_HA']
            .transform(lambda x: x.rolling(window=3, min_periods=1).mean())
        )

        # Rasio efisiensi
        df['Efisiensi_Pupuk'] = df['Hasil_Panen_Ton_per_HA'] / (df['Penggunaan_Pupuk_KG_per_HA'] + 1e-6)
        df['Efisiensi_Pestisida'] = df['Hasil_Panen_Ton_per_HA'] / (df['Penggunaan_Pestisida_KG_per_HA'] + 1e-6)
        df['Efisiensi_Air'] = df['Hasil_Panen_Ton_per_HA'] / (df['Total_Curah_Hujan_mm'] + 1e-6)
        df['CO2_per_Hasil_Panen'] = df['Emisi_CO2_JT_Ton'] / (df['Hasil_Panen_Ton_per_HA'] + 1e-6)
            
        # Fitur Interaksi
        df['Suhu_x_CurahHujan'] = df['Suhu_Rata_Rata_C'] * df['Total_Curah_Hujan_mm'] 
        df['Pupuk_x_Pestisida'] = df['Penggunaan_Pupuk_KG_per_HA'] * df['Penggunaan_Pestisida_KG_per_HA']
        df['AksesIrigasi_x_CurahHujan'] = df['Akses_Irigasi'] * df['Total_Curah_Hujan_mm']
        df['Suhu_x_CO2'] = df['Suhu_Rata_Rata_C'] * df['Emisi_CO2_JT_Ton']

        # Threshold berdasarkan persentil global
        rain_threshold = df['Total_Curah_Hujan_mm'].quantile(0.9)
        temp_threshold = df['Suhu_Rata_Rata_C'].quantile(0.9)
        soil_threshold = df['Indeks_Kesehatan_Tanah'].quantile(0.1)

        df['CurahHujan_Ekstrim'] = (df['Total_Curah_Hujan_mm'] > rain_threshold).astype(int)
        df['Suhu_Ekstrim'] = (df['Suhu_Rata_Rata_C'] > temp_threshold).astype(int)
        df['Kesehatan_Tanah_Buruk'] = (df['Indeks_Kesehatan_Tanah'] < soil_threshold).astype(int)
        
        return df


# ==== Feature Engineering untuk Predict (tanpa target) ====
class FeatureEngineeringPredict(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.temp_threshold = None
        self.rain_threshold = None
        self.soil_threshold = None
    
    def fit(self, X, y=None):
        # Simpan threshold jika ada kolom Suhu di data (untuk data train)
        if 'Suhu_Rata_Rata_C' in X.columns:
            self.temp_threshold = X['Suhu_Rata_Rata_C'].quantile(0.9)
        return self
    
    def transform(self, X):
        df = X.copy()

        # Kolom gabungan wilayah
        df['Negara_Wilayah'] = df['Nama_Negara'] + '_' + df['Wilayah']

        # Representasi waktu linear
        df['Laju_Pertumbuhan_Tahun'] = df['Tahun'] - df['Tahun'].min()

        # Dekade
        df['Dekade'] = (df['Tahun'] // 10) * 10

        # Perubahan hasil panen per tahun (masih bisa, karena tidak pakai target)
        df['Delta_Hasil_Panen'] = df.groupby(['Nama_Negara', 'Jenis_Tanaman'])['Hasil_Panen_Ton_per_HA'].diff()
        
        # Moving average hasil panen 3 tahun terakhir
        df['Moving_Avg_Hasil_Panen_3Tahun'] = (
            df.groupby(['Nama_Negara', 'Jenis_Tanaman'])['Hasil_Panen_Ton_per_HA']
            .transform(lambda x: x.rolling(window=3, min_periods=1).mean())
        )

        # Rasio efisiensi (tidak pakai target)
        df['Efisiensi_Pupuk'] = df['Hasil_Panen_Ton_per_HA'] / (df['Penggunaan_Pupuk_KG_per_HA'] + 1e-6)
        df['Efisiensi_Pestisida'] = df['Hasil_Panen_Ton_per_HA'] / (df['Penggunaan_Pestisida_KG_per_HA'] + 1e-6)
        df['Efisiensi_Air'] = df['Hasil_Panen_Ton_per_HA'] / (df['Total_Curah_Hujan_mm'] + 1e-6)
        df['CO2_per_Hasil_Panen'] = df['Emisi_CO2_JT_Ton'] / (df['Hasil_Panen_Ton_per_HA'] + 1e-6)

        # Fitur interaksi — hanya yang tidak melibatkan target
        df['Pupuk_x_Pestisida'] = df['Penggunaan_Pupuk_KG_per_HA'] * df['Penggunaan_Pestisida_KG_per_HA']
        df['AksesIrigasi_x_CurahHujan'] = df['Akses_Irigasi'] * df['Total_Curah_Hujan_mm']

        # Fitur suhu tidak tersedia → isi dummy agar struktur tetap sama
        df['Perubahan_Suhu_Tahunan'] = 0
        df['Suhu_x_CurahHujan'] = 0
        df['Suhu_x_CO2'] = 0
        df['Suhu_Ekstrim'] = 0

        # Threshold berdasarkan global train (asumsi nilai dummy di sini)
        df['CurahHujan_Ekstrim'] = (df['Total_Curah_Hujan_mm'] > df['Total_Curah_Hujan_mm'].quantile(0.9)).astype(int)
        df['Kesehatan_Tanah_Buruk'] = (df['Indeks_Kesehatan_Tanah'] < df['Indeks_Kesehatan_Tanah'].quantile(0.1)).astype(int)

        return df

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8000 entries, 295 to 6224
Data columns (total 30 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Tahun                           8000 non-null   int64  
 1   Nama_Negara                     8000 non-null   object 
 2   Wilayah                         8000 non-null   object 
 3   Jenis_Tanaman                   8000 non-null   object 
 4   Total_Curah_Hujan_mm            8000 non-null   float64
 5   Emisi_CO2_JT_Ton                8000 non-null   float64
 6   Hasil_Panen_Ton_per_HA          8000 non-null   float64
 7   Kejadian_Cuaca_Ekstrim          8000 non-null   int64  
 8   Akses_Irigasi                   8000 non-null   float64
 9   Penggunaan_Pestisida_KG_per_HA  8000 non-null   float64
 10  Penggunaan_Pupuk_KG_per_HA      8000 non-null   float64
 11  Indeks_Kesehatan_Tanah          8000 non-null   float64
 12  Strategi_Adaptasi               8000 

In [8]:
df_test

,ID,Tahun,Nama_Negara,Jenis_Tanaman,Total_Curah_Hujan_mm,Emisi_CO2_JT_Ton,Hasil_Panen_Ton_per_HA,Kejadian_Cuaca_Ekstrim,Akses_Irigasi,Penggunaan_Pestisida_KG_per_HA,Penggunaan_Pupuk_KG_per_HA,Indeks_Kesehatan_Tanah,Strategi_Adaptasi,Negara_Wilayah
0,8000,2022,Russia,Fruits,1173.04,10.29,2.440,3,59.17,29.17,80.06,74.93,Tanpa Adaptasi,Russia_Central
1,8001,1993,Brazil,Rice,310.08,13.90,3.276,2,60.59,4.76,70.63,97.59,Tanpa Adaptasi,Brazil_South
2,8002,1998,India,Soybeans,1348.17,12.90,2.620,3,90.93,5.95,63.87,74.81,Tanaman Tahan Kekeringan,India_Maharashtra
3,8003,2010,Russia,Soybeans,1732.43,23.82,3.717,4,35.11,7.49,81.81,87.01,Rotasi Tanaman,Russia_Volga
4,8004,1998,Canada,Sugarcane,2242.77,28.92,2.034,6,68.75,43.35,48.14,64.22,Manajemen Air,Canada_British Columbia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,9995,1993,Nigeria,Cotton,2987.93,12.82,2.040,3,54.54,2.18,75.93,61.03,Manajemen Air,Nigeria_North West
1996,9996,2000,Brazil,Corn,878.80,2.82,1.920,8,91.63,7.15,50.57,61.61,Pertanian Organik,Brazil_North
1997,9997,2023,China,Fruits,1858.37,29.98,1.926,2,72.31,46.20,22.35,52.12,Tanaman Tahan Kekeringan,China_East
1998,9998,1999,Nigeria,Soybeans,2307.96,0.99,0.960,4,36.41,21.31,9.13,47.08,Rotasi Tanaman,Nigeria_North West


In [22]:
# Inisialisasi variabel dan list negara
negara_list = df['Nama_Negara'].unique()
results = []
models_dict = {}  # Simpan model per negara

In [ ]:
# Loop model per negara
for negara in negara_list:
    print(f"\nTraining model untuk negara: {negara}")

    # Subset per negara
    subset = df[df['Nama_Negara'] == negara].copy()

    fe_train = FeatureEngineeringTrain()
    subset = fe_train.fit_transform(subset)

    # Pisahkan X dan y
    X = subset.drop(columns=['Suhu_Rata_Rata_C'])
    y = subset['Suhu_Rata_Rata_C']

    # Tentukan kolom
    cat_target_enc = ['Negara_Wilayah']
    cat_onehot = ['Jenis_Tanaman', 'Strategi_Adaptasi', 'Nama_Negara']
    num_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

    # Pastikan kolom numerik tidak double (hapus ID atau Tahun jika mau)
    num_cols = [col for col in num_cols if col not in ['ID']]  # opsional

    # Preprocessor
    preprocessor = ColumnTransformer([
        ('target_enc', TargetEncoder(), cat_target_enc),
        ('onehot', OneHotEncoder(handle_unknown='ignore'), cat_onehot),
        ('scaler', StandardScaler(), num_cols)
    ])

    # Pipeline
    train_pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('algo', xgb.XGBRegressor())
    ])

    params = {
    'algo__n_estimators': np.arange(100, 1000, 100),
    'algo__learning_rate': np.linspace(0.0001, 0.01, 5),
    'algo__max_depth': np.arange(3, 12, 1),
    'algo__min_child_weight': np.arange(1, 10, 1),
    'algo__subsample': np.linspace(0.6, 1.0, 5),
    'algo__colsample_bytree': np.linspace(0.6, 1.0, 5),
    'algo__gamma': np.linspace(0, 1, 10),
    'algo__reg_alpha': np.logspace(-3, 2, 6),   # 0.001 – 100
    'algo__reg_lambda': np.logspace(-3, 2, 6)
    }
    model = RandomizedSearchCV(
    estimator=train_pipeline,
    param_distributions=params,
    n_iter=40,
    scoring='neg_root_mean_squared_error',
    cv=5, 
    n_jobs=-1,
    verbose=2,
    random_state=42,
    refit=True
    )

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    # Train
    model.fit(X_train, y_train)

    # Prediksi
    y_pred = model.predict(X_test)

    # Evaluasi
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

    results.append({
        'Negara': negara,
        'RMSE': rmse,
        'MAE': mae,
        'MAPE (%)': mape
    })
    
    # Simpan model terbaik per negara
    models_dict[negara] = model.best_estimator_



Training model untuk negara: USA
Fitting 5 folds for each of 40 candidates, totalling 200 fits
[CV] END algo__colsample_bytree=1.0, algo__gamma=0.8888888888888888, algo__learning_rate=0.01, algo__max_depth=10, algo__min_child_weight=6, algo__n_estimators=400, algo__reg_alpha=0.01, algo__reg_lambda=100.0, algo__subsample=0.6; total time=   1.3s
[CV] END algo__colsample_bytree=1.0, algo__gamma=0.8888888888888888, algo__learning_rate=0.01, algo__max_depth=10, algo__min_child_weight=6, algo__n_estimators=400, algo__reg_alpha=0.01, algo__reg_lambda=100.0, algo__subsample=0.6; total time=   1.3s
[CV] END algo__colsample_bytree=1.0, algo__gamma=0.8888888888888888, algo__learning_rate=0.01, algo__max_depth=10, algo__min_child_weight=6, algo__n_estimators=400, algo__reg_alpha=0.01, algo__reg_lambda=100.0, algo__subsample=0.6; total time=   1.3s
[CV] END algo__colsample_bytree=1.0, algo__gamma=0.8888888888888888, algo__learning_rate=0.01, algo__max_depth=10, algo__min_child_weight=6, algo__n_es

In [25]:
# Hasil Evaluasi per Negara
hasil = pd.DataFrame(results).sort_values(by='RMSE')
print("\nHasil Evaluasi per Negara:")
print(hasil)


Hasil Evaluasi per Negara:
      Negara      RMSE       MAE   MAPE (%)
4     Canada  1.109252  0.785900  11.040223
8  Argentina  1.217706  0.900688  10.317659
7     France  1.252419  0.863223  15.077551
9     Brazil  1.322675  0.986039  29.486785
1      China  1.344099  0.948378  12.395504
0        USA  1.378688  0.958033  14.524797
5  Australia  1.395736  0.959935  16.414077
2      India  1.489196  0.964744   9.191909
3    Nigeria  1.550441  0.984277  11.689697
6     Russia  1.609112  1.087755  10.721306


In [ ]:
# Load data test
df_test_full = pd.read_csv('/Users/macbookpro/Documents/Documents/open_project/compe/Intelecta/test_cleaned.csv')

# Feature engineering untuk test
fe_predict = FeatureEngineeringPredict()
df_test_fe = fe_predict.fit_transform(df_test_full)

# List untuk menyimpan prediksi
all_predictions = []

# Prediksi per negara
for negara in negara_list:
    print(f"Prediksi untuk negara: {negara}")
    
    # Filter data test untuk negara ini
    df_test_negara = df_test_fe[df_test_fe['Nama_Negara'] == negara].copy()
    
    if len(df_test_negara) == 0:
        print(f"  ⚠️ Tidak ada data test untuk {negara}")
        continue
    
    # Ambil model untuk negara ini
    model_negara = models_dict[negara]
    
    # Prediksi
    y_pred_negara = model_negara.predict(df_test_negara)
    
    # Simpan dengan index asli
    df_test_negara['Suhu_Rata_Rata_C_Pred'] = y_pred_negara
    all_predictions.append(df_test_negara[['Suhu_Rata_Rata_C_Pred']])
    
    print(f"  ✓ {len(y_pred_negara)} prediksi dibuat")

# Gabungkan semua prediksi
df_predictions = pd.concat(all_predictions)

# Urutkan sesuai index asli (order dari df_test_full)
df_predictions = df_predictions.reindex(df_test_fe.index)

# Hasil prediksi final
y_pred_new = df_predictions['Suhu_Rata_Rata_C_Pred'].values

y_pred_new

Prediksi untuk negara: USA
  ✓ 201 prediksi dibuat
Prediksi untuk negara: China
  ✓ 208 prediksi dibuat
Prediksi untuk negara: India
  ✓ 207 prediksi dibuat
Prediksi untuk negara: Nigeria
  ✓ 203 prediksi dibuat
Prediksi untuk negara: Canada
  ✓ 183 prediksi dibuat
Prediksi untuk negara: Australia
  ✓ 206 prediksi dibuat
Prediksi untuk negara: Russia
  ✓ 190 prediksi dibuat
Prediksi untuk negara: France
  ✓ 207 prediksi dibuat
Prediksi untuk negara: Argentina
  ✓ 212 prediksi dibuat
Prediksi untuk negara: Brazil
  ✓ 183 prediksi dibuat


array([ 0.24512221,  0.5662661 ,  0.12730153, ...,  0.23156336,
        0.76489043, -0.05642051], dtype=float32)

In [29]:
len(y_pred_new)

2000

In [30]:
X_test_id = pd.read_csv('/Users/macbookpro/Documents/Documents/open_project/compe/Intelecta/intelecta-cup-data-mining-competition/sample_submission.csv')
X_test_id = X_test_id.ID
X_test_id

0       8000
1       8001
2       8002
3       8003
4       8004
        ... 
1995    9995
1996    9996
1997    9997
1998    9998
1999    9999
Name: ID, Length: 2000, dtype: int64

In [31]:
submission = pd.DataFrame({
    'ID': X_test_id,
    'Suhu_Rata_Rata_C': y_pred_new
})
submission

,ID,Suhu_Rata_Rata_C
0,8000,0.245122
1,8001,0.566266
2,8002,0.127302
3,8003,0.252705
4,8004,0.207048
...,...,...
1995,9995,0.254261
1996,9996,0.250445
1997,9997,0.231563
1998,9998,0.764890


In [32]:
submission.to_csv('PANDORA_submission_8.csv', index=False, encoding='utf-8')

In [14]:
fe_test = FeatureEngineeringPredict()
new_data_fe = fe_test.transform(df_new)

In [ ]:

# (Opsional) Simpan model per negara
import joblib
for negara, model_data in zip(hasil['Negara'], results):
    subset = df[df['Nama_Negara'] == negara].copy()
    X = subset.drop(columns=['Suhu_Rata_Rata_C'])
    y = subset['Suhu_Rata_Rata_C']

    # Train full data negara tersebut
    pipeline.fit(X, y)
    joblib.dump(pipeline, f'model_{negara}.pkl')

print("\nSemua model negara berhasil disimpan!")

In [ ]:
df_USA = df[df['Nama_Negara'] == 'USA'].copy()